In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [60]:
# Tiny Shakespear
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-22 10:31:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  6.29MB/s    in 0.2s    

2025-12-22 10:31:58 (6.29 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [61]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911

od nos CAy go ghanoray t, co haringoudrou clethe k,LARof fr werar,
Is fa!


Thilemel cia h hmboomyorarifrcitheviPO, tle dst f qur'dig t cof boddo y t o ar pileas h mo wierl t,
S:
STENENEat I athe thounomy tinrent distesisanimald 3I: eliento ald, avaviconofrisist me Busarend un'soto vat s k,
SBRI he the f wendleindd t acoe ts ansu, thy ppr h.QULY:
KIIsqu pr odEd ch,
APrnes ouse bll owhored miner t ooon'stoume bupromo! fifoveghind hiarnge s.
MI aswimy or m, wardd tw'To tee abifewoetsphin sed The a


# Mathematical Trick in self-attention models

In [62]:
# considering the following toy example:
torch.manual_seed(1337)
B,T,C= 4,8,2 #batch, time, channels(information at each point in the sequence)
x= torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

##### tokens should not interact with other future tokens, it should only talk to the previous tokens so the information flows from previous context to the current time stamp.


In [74]:
# We want x[b,t]=mean_{i<=t} x[b,i]
#bow means bag of words
xbow= torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev= x[b, :t+1] #(t,C)
        xbow[b,t]= torch.mean(xprev, 0)

In [78]:
wei= torch.tril(torch.ones(T,T))
wei= wei/wei.sum(1, keepdim=True)
xbow2= wei @ x # (B,T,T) @ (B,T,C) ---> (B,T,C)

In [77]:
torch.allclose(xbow, xbow2, atol=1e-6)

True

In [94]:
tril = torch.tril(torch.ones(T,T))
wei= torch.zeros((T,T))
wei= tril.masked_fill(tril == 0, float('-inf'))
wei= F.softmax(wei, dim=-1)
xbow3= wei @ x 
torch.allclose(xbow, xbow3, atol=1e-6)

True

This code computes a **bag-of-words (BoW) style running average** of the input tensor `x`. For each batch index `b` and time step `t`, the value `xbow[b, t]` is set to the mean of all vectors `x[b, i]` where `i ≤ t`. In other words, each position contains the average of the current and all previous token representations. This corresponds to the definition  

$$
x[b,t] = \text{mean}_{   i \le } x[b,i]
$$

and is referred to as a *bag-of-words* representation because it aggregates information from previous tokens without preserving their order beyond inclusion.


In [66]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [67]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

`xbow[0]` is printing with the average of all the past context, <br>Whereas `x[0]` is only considering the present element.

But I don't think any of these is efficient.(#freewill)


In [68]:
torch.manual_seed(42)
a= torch.ones(3,3)
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [69]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [70]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
a= a/torch.sum(a,1, keepdim=True)
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
